In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np

In [4]:
def load_images(datasets):
    datagen=ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    data_load=datagen.flow_from_directory(
    datasets,
    target_size=(224,224),
    batch_size=16,
    class_mode="binary")
    return data_load

In [5]:
datasets_ds="Ds"
data_ds=load_images(datasets_ds)
base_model=ResNet50(weights="imagenet",include_top=False)

Found 160 images belonging to 2 classes.


94765736/94765736 [==============================] - 38s 0us/step


In [14]:


def extract_features(generator,sample_count):
    features=np.zeros(shape=(sample_count,7,7,2048))
    labels=  np.zeros(shape=(sample_count))  
    i=0
    for input_patch,label_patch in generator:
        model=base_model.predict(input_patch)
        features[i*16:(i+1)*16]=model
        labels[i*16:(i+1)*16]=label_patch
        i+=1
        if (i*16)>=sample_count:
            break
    
    return features,labels

    
    
    
    
    

In [15]:
x,y=extract_features(data_ds,data_ds.samples)

1/1 [==============================] - 1s 1s/step


In [20]:
x_features=x.reshape(x.shape[0],-1)

7*7*2048,x_features.shape

(100352, (160, 100352))

In [25]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest= train_test_split(x_features,y,test_size=0.2,random_state=30)

In [27]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
svc=SVC(kernel="linear")
svc.fit(xtrain,ytrain)
predict=svc.predict(xtest)
svm_accuracy=accuracy_score(predict,ytest)
svm_accuracy,classification_report(predict,ytest)

(1.0,
 '              precision    recall  f1-score   support\n\n         0.0       1.00      1.00      1.00        26\n         1.0       1.00      1.00      1.00         6\n\n    accuracy                           1.00        32\n   macro avg       1.00      1.00      1.00        32\nweighted avg       1.00      1.00      1.00        32\n')